#### **Partie 1: Importation des Bibliothèques et Configuration Initiale**

In [ ]:
# Importer les bibliothèques nécessaires
from google.colab import drive
import shutil
import torch
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset



**Explication:**
- **google.colab.drive** : Permet de monter Google Drive pour sauvegarder les résultats.
- **shutil** : Pour copier des fichiers et dossiers.
- **torch** : Utilisé pour les opérations sur le GPU avec PyTorch.
- **pandas** : Manipule les données sous forme de DataFrame.
- **re** : Pour les opérations de traitement de texte.
- **sklearn.model_selection.train_test_split** : Pour diviser les données en ensembles d'entraînement et de validation.
- **transformers** : Fournit des outils pour le modèle T5 et la gestion de l'entraînement.
- **datasets** : Permet de charger des datasets à partir de fichiers CSV.

---

#### **Partie 2: Chargement et Préparation des Données**

In [ ]:
# Monter Google Drive
drive.mount('/content/drive')

# Charger les données à partir du fichier JSON
df2 = pd.read_json("hf://datasets/ltg/en-wiki-paraphrased/train.jsonl", lines=True)

# Sauvegarder les données en CSV
csv_file = 'en_wiki_paraphrased.csv'
df2.to_csv(csv_file, index=False)

# Charger les données CSV
df1 = pd.read_csv(csv_file)
df = df1.head(100000)

# Diviser les données en ensembles d'entraînement et de validation
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

# Sauvegarder les ensembles en fichiers CSV
output_file1 = 'train.csv'
output_file2 = 'val.csv'
df_train.to_csv(output_file1, index=False)
df_val.to_csv(output_file2, index=False)



**Explication:**
- Les données sont chargées à partir d'un fichier JSON en utilisant `pandas`.
- Les données sont ensuite sauvegardées au format CSV.
- Le CSV est divisé en ensembles d'entraînement et de validation en utilisant `train_test_split`.
- Les ensembles divisés sont sauvegardés en fichiers CSV distincts.

---

#### **Partie 3: Chargement des Datasets et Prétraitement**


In [ ]:
# Charger les datasets à partir des fichiers CSV
dataset_train = load_dataset('csv', data_files='train.csv', split='train')
dataset_valid = load_dataset('csv', data_files='val.csv', split='train')

# Initialiser le tokenizer
tokenizer = T5Tokenizer.from_pretrained(MODEL)

def preprocess_function(examples):
    inputs = [f" paraphrase : {original}" for (original) in zip(examples['original'])]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )

    cleaned_tag = examples['paraphrase']
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            cleaned_tag,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Appliquer la fonction de prétraitement aux datasets
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS  # Utiliser un seul processus
)
tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS  # Utiliser un seul processus
)



**Explication:**
- Les datasets CSV sont chargés en utilisant `load_dataset`.
- Le tokenizer T5 est initialisé pour transformer le texte.
- La fonction `preprocess_function` est définie pour ajouter les balises nécessaires et tronquer/padd les textes.
- La fonction de prétraitement est appliquée aux datasets d'entraînement et de validation.

---


#### **Partie 4: Initialisation du Modèle et Arguments d'Entraînement**

In [ ]:
# Initialiser le modèle T5
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    eval_strategy='steps',  # Déprécié, à remplacer par `eval_steps` dans la version future
    save_steps=2000,
    eval_steps=1000,  # Évalue la perte de validation tous les 100 pas
    load_best_model_at_end=True,
    save_total_limit=5,
    report_to='tensorboard',
    learning_rate=0.0001,
    fp16=True,
    dataloader_num_workers=0  # Réduire le nombre de travailleurs
)



**Explication:**
- Le modèle T5 est initialisé et déplacé sur le GPU si disponible.
- Les arguments pour l'entraînement sont définis, y compris le nombre d'époques, la taille des lots, et la stratégie de sauvegarde.

---


#### **Partie 5: Entraînement du Modèle**


In [ ]:
# Entraîner le modèle
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,  # Dataset de validation (test)
)

history = trainer.train()



**Explication:**
- Le `Trainer` de la bibliothèque `transformers` est utilisé pour entraîner le modèle sur les données préparées.

---


#### **Partie 6: Sauvegarde du Tokenizer et du Modèle**

In [ ]:
# Sauvegarder le tokenizer
tokenizer.save_pretrained(OUT_DIR)

# Copier les résultats vers Google Drive
source_folder = f'./{OUT_DIR}'
target_folder = f'/content/drive/My Drive/T5vfin/{OUT_DIR}'
shutil.copytree(source_folder , target_folder)



**Explication:**
- Le tokenizer est sauvegardé dans le répertoire spécifié.
- Les résultats sont copiés dans Google Drive pour la sauvegarde.

---

#### **Partie 7: Chargement du Modèle Sauvegardé et Fonction de Paraphrasage**

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Charger le modèle et le tokenizer sauvegardés
model_path = './T5vfin/res_T5small/checkpoint-15000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained('./T5vfin/res_T5small')

def paraphraser(text, model, tokenizer):
    input_text = f" paraphrase : {text}"
    inputs = tokenizer.encode(
        input_text,
        return_tensors='pt',
        max_length=256,
        padding='max_length',
        truncation=True
    )

    # Générer la paraphrase
    corrected_ids = model.generate(
        inputs,
        max_length=256,
        num_beams=5,
        early_stopping=True
    )

    # Décoder la paraphrase
    paraphrase = tokenizer.decode(
        corrected_ids[0],
        skip_special_tokens=True
    )
    return paraphrase

# Exemple de texte à paraphraser
text = """
Des averses orageuses localement fortes et de fortes rafales de vent avec chasse-poussières locales sont prévues dimanche dans plusieurs provinces du Royaume, a annoncé la Direction générale de la météorologie (DGM). ...
"""

print(paraphraser(text, model, tokenizer))



**Explication:**
- Le modèle et le tokenizer sauvegardés sont chargés.
- La fonction `paraphraser` est définie pour générer des paraphrases du texte fourni.
- Un exemple de texte est paraphrasé en utilisant le modèle.

---